**Imports and Settings**

In [1]:
# Import stuff
import os
from os import walk
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from sklearn.preprocessing import MinMaxScaler
import glob
import seaborn as sns; sns.set()
from numpy import inf
from scipy.signal import hilbert, butter, filtfilt
from scipy.fftpack import fft,fftfreq,rfft,irfft,ifft
import seaborn as sns
import scipy.stats as stats
from matplotlib.backends.backend_pdf import PdfPages
#if you run it locally, you should install tqdm package
from tqdm import tqdm
from scipy.io import loadmat

#import tqdm

In [2]:
#set the figure size
from pylab import *
rcParams['figure.figsize'] = 20,5

*Loop through subjects and obtain EEG data in epochs of interest*

In [3]:
# Get all trial start timestamps
df_timestamps = pd.read_csv('timing_match.csv')

In [4]:
df_timestamps

,Event,Marker,OpenBCI-RAW-2019-06-18_14-55-30.txt,OpenBCI-RAW-2019-06-20_16-06-23.txt,OpenBCI-RAW-2019-06-20_16-48-25.txt,OpenBCI-RAW-2019-06-21_15-48-38.txt,OpenBCI-RAW-2019-06-24_12-31-44.txt,OpenBCI-RAW-2019-06-21_16-12-59.txt,OpenBCI-RAW-2019-06-25_15-35-22.txt,OpenBCI-RAW-2019-06-25_16-40-28.txt,...,OpenBCI-RAW-2019-06-28_12-02-45.txt,OpenBCI-RAW-2019-07-01_10-19-46.txt,OpenBCI-RAW-2019-07-01_10-45-04.txt,OpenBCI-RAW-2019-07-01_12-20-07.txt,OpenBCI-RAW-2019-07-01_12-47-43.txt,OpenBCI-RAW-2019-07-01_15-48-56.txt,OpenBCI-RAW-2019-07-02_10-20-09.txt,OpenBCI-RAW-2019-07-02_11-20-05.txt,OpenBCI-RAW-2019-07-02_11-47-52.txt,OpenBCI-RAW-2019-07-02_16-12-07.txt
0,EyesOpenedStart,1,2019-06-18 15:06:02.734,2019-06-20 16:36:19.984,2019-06-20 16:52:25.726,2019-06-21 16:02:28.742,2019-06-24 13:08:44.601,2019-06-24 12:19:39.570,2019-06-25 15:56:37.015,2019-06-25 17:02:22.679,...,2019-06-28 12:16:50.593,2019-07-01 10:35:12.898,2019-07-01 11:19:47.945,2019-07-01 12:38:05.117,2019-07-01 15:10:26.609,2019-07-01 16:03:54.539,2019-07-02 10:32:40.687,2019-07-02 11:37:26.109,2019-07-02 12:26:01.242,2019-07-02 16:20:01.835
1,EyesOpenedEnd,2,2019-06-18 15:06:18.000,2019-06-20 16:36:35.226,2019-06-20 16:52:40.898,2019-06-21 16:02:43.882,2019-06-24 13:08:59.875,2019-06-24 12:19:54.820,2019-06-25 15:56:52.289,2019-06-25 17:02:37.859,...,2019-06-28 12:17:05.804,2019-07-01 10:35:28.210,2019-07-01 11:20:03.187,2019-07-01 12:38:20.304,2019-07-01 15:10:41.781,2019-07-01 16:04:09.726,2019-07-02 10:32:55.906,2019-07-02 11:37:41.335,2019-07-02 12:26:16.523,2019-07-02 16:20:16.984
2,EyesClosedStart,3,2019-06-18 15:06:22.929,2019-06-20 16:36:40.140,2019-06-20 16:52:45.914,2019-06-21 16:02:48.898,2019-06-24 13:09:04.843,2019-06-24 12:19:59.726,2019-06-25 15:56:57.195,2019-06-25 17:02:42.828,...,2019-06-28 12:17:10.726,2019-07-01 10:35:33.210,2019-07-01 11:20:08.093,2019-07-01 12:38:25.187,2019-07-01 15:10:46.718,2019-07-01 16:04:14.664,2019-07-02 10:33:00.890,2019-07-02 11:37:46.273,2019-07-02 12:26:21.429,2019-07-02 16:20:21.984
3,EyesClosedEnd,4,2019-06-18 15:06:38.148,2019-06-20 16:36:55.359,2019-06-20 16:53:01.070,2019-06-21 16:03:04.085,2019-06-24 13:09:20.023,2019-06-24 12:20:14.984,2019-06-25 15:57:12.390,2019-06-25 17:02:57.992,...,2019-06-28 12:17:25.976,2019-07-01 10:35:48.375,2019-07-01 11:20:23.343,2019-07-01 12:38:40.468,2019-07-01 15:11:01.921,2019-07-01 16:04:29.921,2019-07-02 10:33:16.132,2019-07-02 11:38:01.406,2019-07-02 12:26:36.656,2019-07-02 16:20:37.109
4,PenaltyKickTrial,5,2019-06-18 15:07:04.640,2019-06-20 16:37:40.156,2019-06-20 16:56:00.242,2019-06-21 16:03:21.335,2019-06-24 13:10:45.281,2019-06-24 12:20:45.898,2019-06-25 15:58:39.578,2019-06-25 17:03:28.257,...,2019-06-28 12:18:28.718,2019-07-01 10:36:05.726,2019-07-01 11:21:08.328,2019-07-01 12:39:04.601,2019-07-01 15:11:30.367,2019-07-01 16:04:57.742,2019-07-02 10:33:38.007,2019-07-02 11:38:34.273,2019-07-02 12:26:55.718,2019-07-02 16:21:01.625
5,PenaltyKickTrial,5,2019-06-18 15:07:14.843,2019-06-20 16:37:50.367,2019-06-20 16:56:10.500,2019-06-21 16:03:31.664,2019-06-24 13:10:59.835,2019-06-24 12:20:56.265,2019-06-25 16:00:48.632,2019-06-25 17:03:38.609,...,2019-06-28 12:18:38.953,2019-07-01 10:36:24.687,2019-07-01 11:21:18.671,2019-07-01 12:39:14.796,2019-07-01 15:11:40.593,2019-07-01 16:05:08.078,2019-07-02 10:33:48.210,2019-07-02 11:38:44.531,2019-07-02 12:27:06.039,2019-07-02 16:21:25.171
6,PenaltyKickTrial,5,2019-06-18 15:07:25.078,2019-06-20 16:38:00.679,2019-06-20 16:56:20.687,2019-06-21 16:03:42.023,2019-06-24 13:11:35.562,2019-06-24 12:21:06.468,2019-06-25 16:00:58.859,2019-06-25 17:03:48.851,...,2019-06-28 12:18:49.250,2019-07-01 10:36:34.898,2019-07-01 11:21:28.929,2019-07-01 12:39:25.117,2019-07-01 15:11:50.937,2019-07-01 16:05:18.289,2019-07-02 10:33:58.546,2019-07-02 11:38:54.765,2019-07-02 12:27:16.257,2019-07-02 16:21:36.124
7,PenaltyKickTrial,5,2019-06-18 15:07:35.398,2019-06-20 16:38:11.007,2019-06-20 16:56:31.015,2019-06-21 16:03:52.335,2019-06-24 13:11:45.835,20

In [5]:
for entry in glob.glob('EEG_openBCI/*.txt'):
    try:
        df = pd.read_csv(entry, skiprows = 6, header = None)
        if pd.isna(df.iloc[-1,-1]) == True or len(df.iloc[-1,-1]) < 12:
            last = df.iloc[-2,-1]
        else:
            last = df.iloc[-1,-1]

        print(entry[12:],df.iloc[0,-1], last )
    except:
        print(entry[12:])

OpenBCI-RAW-2019-06-13_16-39-24.txt  16:46:14.566  16:47:07.530
OpenBCI-RAW-2019-06-17_16-16-35.txt  16:23:54.316  17:02:31.492
OpenBCI-RAW-2019-06-18_14-36-41.txt  14:38:45.386  14:46:02.850
OpenBCI-RAW-2019-06-18_14-46-19.txt  14:47:01.979  14:50:34.642
OpenBCI-RAW-2019-06-18_14-51-48.txt  14:52:20.311  14:55:03.453
OpenBCI-RAW-2019-06-18_14-55-30.txt  14:55:54.124  15:16:59.295
OpenBCI-RAW-2019-06-20_16-06-23.txt  16:29:44.311  16:48:19.526
OpenBCI-RAW-2019-06-20_16-48-25.txt  16:55:40.812  17:06:17.929
OpenBCI-RAW-2019-06-21_15-09-23.txt
OpenBCI-RAW-2019-06-21_15-48-38.txt  15:59:53.433  16:12:51.131
OpenBCI-RAW-2019-06-21_16-12-59.txt  12:17:58.338  12:31:41.529
OpenBCI-RAW-2019-06-24_12-31-44.txt  13:07:49.435  13:20:19.169
OpenBCI-RAW-2019-06-24_13-20-21.txt  13:32:34.837  13:38:03.113
OpenBCI-RAW-2019-06-25_15-35-22.txt  15:42:00.745  16:09:28.585
OpenBCI-RAW-2019-06-25_16-29-21.txt  16:32:28.352  16:34:44.883
OpenBCI-RAW-2019-06-25_16-40-28.txt  16:41:31.914  17:13:11.814
Open

In [22]:
def good_epochs(timestamps, df, file):
    good_epoch = 0
    output_folder = 'good_epochs\\'
    # Eyes Open
    print('eo')
    #print(timestamps)
    padding = datetime.timedelta(seconds=10)

    start_time = timestamps[0] - padding
    print(start_time, timestamps[0], padding)
    end_time = timestamps[1] + padding
    raw_duration = end_time - start_time
    df_epoch = df[df['timestamp'].between(start_time, end_time)]
    expected_length = (raw_duration.total_seconds()) * fs
    real_length = len(df_epoch)
    diff_len = abs(real_length - expected_length)/expected_length
    print('real, expected', real_length, expected_length, diff_len)

    if diff_len < 0.03:
        df_epoch.to_csv(output_folder + file[0:-4] + '_' + 'eo.csv', index = False)

    
    # Eyes Closed
    print('ec')

    start_time = timestamps[2] - padding
    end_time = timestamps[3] + padding
    raw_duration = end_time - start_time
    df_epoch = df[df['timestamp'].between(start_time, end_time)]
    expected_length = (raw_duration.total_seconds()) * fs
    real_length = len(df_epoch)
    diff_len = abs(real_length - expected_length)/expected_length
    print('real, expected', real_length, expected_length, diff_len)

    if diff_len < 0.03:
        df_epoch.to_csv(output_folder + file[0:-4] + '_' + 'ec.csv', index = False)

    # Loop through trials
    trial_timestamps = timestamps[4:-1]
    for trial, trial_time in enumerate(trial_timestamps):
        start_time = trial_time - padding
        end_time = trial_time + padding
        expected_duration = end_time - start_time
        print('start, end', start_time, end_time, padding)
        #try:
        df_epoch=df[df['timestamp'].between(start_time, end_time)]
        #df_lengths.append(len(df_epoch))
        #print(df_epoch.head(2))
        raw_duration =  start_time - end_time
        diff_len = abs(expected_duration - raw_duration)/expected_duration
        #print('expected, raw', expected_duration, raw_duration)
        diffs = df_epoch['timestamp'].diff()
        maxdiff = max(diffs[1:]) # max difference excluding first entry NaT
        #print(maxdiff)
        maxdelta = datetime.timedelta(seconds=0.5)
        totaldelta = raw_duration - 2*padding

        print('maxdelta', maxdelta)
        # Set criteria for epoch rejection
        if (totaldelta > maxdelta == True) or (maxdiff > maxdelta) or (diff_len < 0.03):
            print('trial criteria not met')
            print(totaldelta, maxdelta, totaldelta>maxdelta, maxdiff>maxdelta, len(df_epoch), min_len, len(df_epoch) < min_len)
            continue

        else:
            print('else')
            #Add 1 to trial number due to 0-indexing
            df_epoch.to_csv(output_folder + file[0:-4] + '_' + str(trial+1) + '.csv', index = False)
            #print(output_folder + file[0:-4] + '_' + str(trial) + '.csv')
            good_epoch += 1
    #except:
            #print("epoch error ", file, trial_time)
            #continue
    #percent_good = good_epoch/len(timestamps)
    #print("percent of good epochs: " + str(percent_good))

In [23]:
df_lengths = []
fs = 200 # sampling frequency
for entry in glob.glob('EEG_openBCI/*.txt')[5:]:
    try:
        file = entry[12:]
        timestamps = df_timestamps[file].dropna()
        # Convert to datetime format
        timestamps = pd.to_datetime(timestamps, format = '%Y-%m-%d %H:%M:%S.%f', errors = 'coerce')
        trial_duration = datetime.timedelta(seconds=4)

        # Read in data
        df_new = pd.read_csv(entry, sep = ',\s+', skiprows = 6, header=None, usecols = [0, 1, 2, 3, 4, 8], 
                             names = ['sample', 'ch1', 'ch2', 'ch3', 'ch4', 'timestamp'], engine='python')
        #print(df_new.head(1))
        # Drop nan rows
        df_na = df_new.dropna()

        # Drop duplicates if ALL data columns values match
        df = df_na.drop_duplicates(subset = ['ch1', 'ch2', 'ch3', 'ch4'], keep = 'last')

        # Convert to datetime format
        df['datetime'] = file[12:22] + df['timestamp']
        df['timestamp'] = pd.to_datetime(df['datetime'], format = '%Y-%m-%d%H:%M:%S.%f', errors = 'coerce')
        #print(df.head(2))
        print('ratio retained', len(df)/len(df_new))
        # Get good epochs and save them
        good_epochs(timestamps, df, file)
    except:
        print('file error: ' + file)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ratio retained 0.3316048331894064
eo
2019-06-18 15:05:52.734000 2019-06-18 15:06:02.734000 0:00:10
real, expected 5840 7053.2 0.172007032269041
ec
real, expected 5958 7043.8 0.15414974871518217
start, end 2019-06-18 15:06:54.640000 2019-06-18 15:07:14.640000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-18 15:07:04.843000 2019-06-18 15:07:24.843000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-18 15:07:15.078000 2019-06-18 15:07:35.078000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-18 15:07:25.398000 2019-06-18 15:07:45.398000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-18 15:07:35.632000 2019-06-18 15:07:55.632000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-18 15:07:45.898000 2019-06-18 15:08:05.898000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-18 15:07:56.109000 2019-06-18 15:08:16.109000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-18 15:08:06.445000 2019-06-18 15:08:26.445000 0:00:10
maxdelta 0:00:00.50

start, end 2019-06-20 16:42:28.796000 2019-06-20 16:42:48.796000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:42:39 2019-06-20 16:42:59 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:42:49.242000 2019-06-20 16:43:09.242000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:42:59.453000 2019-06-20 16:43:19.453000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:43:09.828000 2019-06-20 16:43:29.828000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:43:20.054000 2019-06-20 16:43:40.054000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:43:30.406000 2019-06-20 16:43:50.406000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:43:40.742000 2019-06-20 16:44:00.742000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:43:50.953000 2019-06-20 16:44:10.953000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-20 16:44:01.257000 2019-06-20 16:44:21.257000 0:00:10
maxdelta 0:00:00.500000

maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:05:14.726000 2019-06-21 16:05:34.726000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:05:25.085000 2019-06-21 16:05:45.085000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:05:35.289000 2019-06-21 16:05:55.289000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:05:45.632000 2019-06-21 16:06:05.632000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:05:55.960000 2019-06-21 16:06:15.960000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:06:06.289000 2019-06-21 16:06:26.289000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:06:16.523000 2019-06-21 16:06:36.523000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:06:26.742000 2019-06-21 16:06:46.742000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:06:37.062000 2019-06-21 16:06:57.062000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-21 16:06:47.398000 2019-06-21 16:0

start, end 2019-06-24 13:17:45.601000 2019-06-24 13:18:05.601000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:17:55.859000 2019-06-24 13:18:15.859000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:18:06.179000 2019-06-24 13:18:26.179000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:18:16.546000 2019-06-24 13:18:36.546000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:18:26.851000 2019-06-24 13:18:46.851000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:18:37.078000 2019-06-24 13:18:57.078000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:18:47.328000 2019-06-24 13:19:07.328000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:18:57.554000 2019-06-24 13:19:17.554000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:19:07.796000 2019-06-24 13:19:27.796000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-06-24 13:19:18 2019-06-24 13:19:38 0:00:10
maxdelta 0:00:00.500000

...

start, end 2019-07-01 11:21:39.421000 2019-07-01 11:21:59.421000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:21:49.710000 2019-07-01 11:22:09.710000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:21:59.921000 2019-07-01 11:22:19.921000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:22:10.273000 2019-07-01 11:22:30.273000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:22:20.601000 2019-07-01 11:22:40.601000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:22:30.851000 2019-07-01 11:22:50.851000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:22:41.164000 2019-07-01 11:23:01.164000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:22:51.484000 2019-07-01 11:23:11.484000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:23:01.781000 2019-07-01 11:23:21.781000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 11:23:12.046000 2019-07-01 11:23:32.046000 0:00:10
maxdelta 

start, end 2019-07-01 15:15:57.523000 2019-07-01 15:16:17.523000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 15:16:07.835000 2019-07-01 15:16:27.835000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 15:16:18.117000 2019-07-01 15:16:38.117000 0:00:10
maxdelta 0:00:00.500000
trial criteria not met
file error: OpenBCI-RAW-2019-07-01_12-47-43.txt
file error: OpenBCI-RAW-2019-07-01_15-20-07.txt
ratio retained 0.9989219204584053
eo
2019-07-01 16:03:44.539000 2019-07-01 16:03:54.539000 0:00:10
real, expected 6838 7037.4 0.028334328018870557
ec
real, expected 7051 7051.4 5.672632385053128e-05
start, end 2019-07-01 16:04:47.742000 2019-07-01 16:05:07.742000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 16:04:58.078000 2019-07-01 16:05:18.078000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 16:05:08.289000 2019-07-01 16:05:28.289000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-01 16:05:18.578000 2019-07-01 16:05:38.578000 0:00:10
ma

maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:37:44.687000 2019-07-02 10:38:04.687000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:37:55.015000 2019-07-02 10:38:15.015000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:38:05.257000 2019-07-02 10:38:25.257000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:38:15.484000 2019-07-02 10:38:35.484000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:38:25.835000 2019-07-02 10:38:45.835000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:38:36.132000 2019-07-02 10:38:56.132000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:38:46.445000 2019-07-02 10:39:06.445000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:38:56.703000 2019-07-02 10:39:16.703000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:39:06.953000 2019-07-02 10:39:26.953000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 10:39:17.195000 2019-07-02 10:3

maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:27:16.492000 2019-07-02 12:27:36.492000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:27:26.734000 2019-07-02 12:27:46.734000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:27:37.070000 2019-07-02 12:27:57.070000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:27:47.343000 2019-07-02 12:28:07.343000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:27:57.679000 2019-07-02 12:28:17.679000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:28:07.960000 2019-07-02 12:28:27.960000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:28:18.179000 2019-07-02 12:28:38.179000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:28:28.406000 2019-07-02 12:28:48.406000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:28:38.656000 2019-07-02 12:28:58.656000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 12:28:48.984000 2019-07-02 12:2

maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:26:47.234000 2019-07-02 16:27:07.234000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:26:57.609000 2019-07-02 16:27:17.609000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:27:07.960000 2019-07-02 16:27:27.960000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:27:18.296000 2019-07-02 16:27:38.296000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:27:28.695000 2019-07-02 16:27:48.695000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:27:38.945000 2019-07-02 16:27:58.945000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:27:49.187000 2019-07-02 16:28:09.187000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:27:59.445000 2019-07-02 16:28:19.445000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:28:09.820000 2019-07-02 16:28:29.820000 0:00:10
maxdelta 0:00:00.500000
else
start, end 2019-07-02 16:28:20.164000 2019-07-02 16:2